# Species classifier based on codon signatire

### Libraries

In [32]:
import pandas as pd

### Dataset

In [33]:
PATH = "/home/zanarfa/proj/codon_signature/data/vibrio_data/"

In [34]:
! ls -l {PATH}

total 668
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 final
-rw-r--r-- 1 zanarfa zanarfa    388 Nov 12 11:10 readme.txt
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 step1
drwxr-xr-x 2 zanarfa zanarfa   4096 Nov 15 15:52 step2
-rw-r--r-- 1 zanarfa zanarfa 648535 Nov 12 11:09 vibrio_original_refseq.csv
-rw-r--r-- 1 zanarfa zanarfa    216 Nov 15 16:16 vibrio_taxid_9.csv
-rw-r--r-- 1 zanarfa zanarfa  10391 Okt 28 17:09 vibrio_taxid.xlsx


#### taxonID/species (as dict and id list)

In [56]:
taxon_species={}

with open (f'{PATH}vibrio_taxid_9.csv', "r") as tax_sp:
    for line in tax_sp:
        line = line.split(",")
        taxonID = line[0]
        species = line[1].replace(" ","_").strip("\n")
        taxon_species[taxonID]=species

In [57]:
taxon_species

{'663': 'Vibrio_alginolyticus',
 '666': 'Vibrio_cholerae',
 '669': 'Vibrio_harveyi',
 '670': 'Vibrio_parahaemolyticus',
 '672': 'Vibrio_vulnificus',
 '676': 'Vibrio_fluvialis',
 '680': 'Vibrio_campbellii',
 '685': 'Vibrio_diazotrophicus',
 '689': 'Vibrio_mediterranei'}

In [58]:
ID_list=[x for x in taxon_species.keys()]

#### data from step 2 as test data (as dictionary of df)

In [71]:
vibrio_originals = {}

for id in ID_list: 
    vibrio_originals[str(taxon_species[id])+"_original"] = pd.read_csv(f'{PATH}step2/{id}_step2_res.tsv', 
                                                                      low_memory=False, sep="\t")

In [72]:
vibrio_originals["Vibrio_mediterranei_original"]

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32
0,0.56,0.5,0.58,0.65,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.88,0.49,0.44,0.97,0.55,0.93,0.99,0.67
1,0.56,0.5,0.57,0.66,0.52,0.76,0.13,0.23,0.39,0.32,...,0.64,0.60,0.87,0.49,0.44,0.97,0.54,0.93,0.99,0.67
2,0.56,0.5,0.58,0.66,0.53,0.76,0.13,0.23,0.39,0.32,...,0.64,0.60,0.87,0.49,0.44,0.97,0.54,0.93,0.99,0.67
3,0.56,0.5,0.57,0.66,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.85,0.49,0.43,0.97,0.54,0.93,0.99,0.67
4,0.57,0.5,0.57,0.66,0.53,0.78,0.13,0.23,0.40,0.33,...,0.64,0.59,0.86,0.49,0.43,0.97,0.54,0.93,0.98,0.67
5,0.56,0.5,0.57,0.66,0.53,0.77,0.13,0.23,0.39,0.32,...,0.64,0.59,0.85,0.49,0.44,0.97,0.54,0.93,0.99,0.66


#### 9 species, 10k lines each (as dictionary of df)

Add a new column with the species

Combine 9 species data

Split training and validation set (90/10)

Split independent and dependent vatiable

Train a random forest on it